# Assignment 5 - Comprehensive - 150 Points

## Learning objectives
By the end of this assignment you should be able to:
* create striped logical volumes on Linux (LVM RAID‑0)
* bind‑mount LVM volumes inside Docker containers
* enable password‑less SSH into bridged network containers
* automate file deployment to multiple hosts with a minimal Ansible playbook
* **(647 graduates only)** use lvm snapshots to recover corrupted files

### Task 1 – Build two striped LVM logical volumes (2 GB each)
Using **four** of the 1 GB disks (pick any four from `/dev/sd[b-f]`) create **two** 2‑disk, striped (RAID‑0) logical volumes of 2 GB each.

* name the volume groups `vgstripe1` and `vgstripe2`  
* name the logical volumes `lvm1` and `lvm2` and make them occupy 100% of the free space in their respective vgs.
* format both volumes with `ext4`  
* mount them at `/mnt/lvm1` and `/mnt/lvm2`

In [9]:
# your commands here (or do it in a terminal)
sudo pvcreate /dev/sdc /dev/sdd /dev/sde /dev/sdf
sudo vgcreate vgstripe1 /dev/sdc /dev/sdd
sudo vgcreate vgstripe2 /dev/sde /dev/sdf
sudo lvcreate -i2 -I64 -l 100%FREE -n lvm1 vgstripe1
sudo lvcreate -i2 -I64 -l 100%FREE -n lvm2 vgstripe2
sudo mkfs.ext4 /dev/vgstripe1/lvm1
sudo mkfs.ext4 /dev/vgstripe2/lvm2
sudo mkdir -p /mnt/lvm1 /mnt/lvm2
sudo mount /dev/vgstripe1/lvm1 /mnt/lvm1
sudo mount /dev/vgstripe2/lvm2 /mnt/lvm2

  Can't initialize physical volume "/dev/sdc" of volume group "vgstripe1" without -ff
  /dev/sdc: physical volume not initialized.
  Can't initialize physical volume "/dev/sdd" of volume group "vgstripe1" without -ff
  /dev/sdd: physical volume not initialized.
  Can't initialize physical volume "/dev/sde" of volume group "vgstripe2" without -ff
  /dev/sde: physical volume not initialized.
  Can't initialize physical volume "/dev/sdf" of volume group "vgstripe2" without -ff
  /dev/sdf: physical volume not initialized.
  A volume group called vgstripe1 already exists.
  A volume group called vgstripe2 already exists.
  Logical volume "lvm1" created.
  Logical volume "lvm2" created.
mke2fs 1.47.0 (5-Feb-2023)
Discarding device blocks: done                            
Creating filesystem with 522240 4k blocks and 130560 inodes
Filesystem UUID: 8b6d80fc-6895-4372-9371-8a7de3292875
Superblock backups stored on blocks: 
	32768, 98304, 163840, 229376, 294912

Allocating group tables: done    

In [10]:
# verification – expect two mounted striped logical volumes, each 2 GB
lsblk -o NAME,SIZE,FSTYPE,MOUNTPOINT | grep -E 'lvm[12]|vgstripe'
df -h | grep '/mnt/lvm'

└─vgstripe1-lvm1     2G ext4        /mnt/lvm1
└─vgstripe1-lvm1     2G ext4        /mnt/lvm1
└─vgstripe2-lvm2     2G ext4        /mnt/lvm2
└─vgstripe2-lvm2     2G ext4        /mnt/lvm2
/dev/mapper/vgstripe1-lvm1  2.0G   24K  1.9G   1% /mnt/lvm1
/dev/mapper/vgstripe2-lvm2  2.0G   24K  1.9G   1% /mnt/lvm2


### Task 2 – Launch two Debian bookworm containers
1. Pull the Docker image `debian:bookworm`.
2. Start two persistent containers named **`debian1`** and **`debian`** using a custom network driver (`-network lvmnet`) that I have given you the command to create in the cell below. Use the "-d" flag to keep them detached, and use "sleep infinity" as the container command so that they stay running in the background. Bind‑mount the host paths `/mnt/lvm1` > `/root/host` (inside **debian1**) and `/mnt/lvm2` > `/root/host` (inside **debian2**).

In [12]:
# Saving you from troubleshooting hell
#
# MTU issues are tricky to troubleshoot because small packets (such as pings) will succeed
# but full size packets (most network traffic) get dropped.
#
# Recall from assignment 1 that your VM is part of a VXLAN that adds 50 to the MTU,
# so we need to drop our MTU to 1450 so that packets fit in the standard 1500 byte limit.
sudo docker network create --driver bridge --opt com.docker.network.driver.mtu=1450 lvmnet

9d764b260e687ac0290e97f78043b6fc6d04802b26ebbdb9b4d09705b3d2ffee


In [13]:
# your commands here (or do it in a terminal)
sudo docker pull debian:bookworm
sudo docker run -d --name debian1 \
  --network lvmnet \
  -v /mnt/lvm1:/root/host \
  debian:bookworm sleep infinity
sudo docker run -d --name debian2 \
  --network lvmnet \
  -v /mnt/lvm2:/root/host \
  debian:bookworm sleep infinity

bookworm: Pulling from library/debian

Digest: sha256:264982ff4d18000fa74540837e2c43ca5137a53a83f8f62c7b3803c0f0bdcd56
Status: Downloaded newer image for debian:bookworm
docker.io/library/debian:bookworm
bc667f44ab3cd8eeab06e6775c11fe6ee17f5875d1ca760c3514d3076d383e71
c3a0a93e089faf41dbaddb5c51b827822d57d3d231718ff05bc7a881e874761c


In [14]:
# verification – containers should be running & mounts visible
sudo docker ps --filter name=debian --format '{{.Names}} -> {{.Status}}'
for c in debian1 debian2; do
  echo "${c} mounts:"
  sudo docker inspect -f '{{ range .Mounts }}{{ .Source }} => {{ .Destination }}{{ println }}{{ end }}' $c
done

debian2 -> Up 5 seconds
debian1 -> Up 6 seconds
debian1 mounts:
/mnt/lvm1 => /root/host

debian2 mounts:
/mnt/lvm2 => /root/host



### Task 3 – Enable SSH access inside each container
Inside both containers, use `apt` to install `openssh-server` and `python3`, start the `sshd` service, and edit `/etc/ssh/sshd_config` to allow root key login.

In [ ]:
# your commands here (or do it in a terminal)
# done in terminal!

In [25]:
# verification – sshd running and sshd_config permits key-based root login
for c in debian1 debian2; do
  echo "=== $c ==="
  sudo docker exec "$c" pgrep -a sshd || echo "sshd NOT running"
  ok_lines=$(sudo docker exec "$c" \
    sh -c "grep -E '^(PermitRootLogin|PubkeyAuthentication)[[:space:]]+yes' /etc/ssh/sshd_config | wc -l")
  if [ "$ok_lines" -eq 2 ]; then
    echo "sshd_config OK (root + key auth enabled)"
  else
    echo "sshd_config missing or incorrectly set directives"
    sudo docker exec "$c" grep -E '^(PermitRootLogin|PubkeyAuthentication)' /etc/ssh/sshd_config
  fi
  echo
done

=== debian1 ===
46 sshd: /usr/sbin/sshd [listener] 0 of 10-100 startups
sshd_config OK (root + key auth enabled)

=== debian2 ===
46 sshd: /usr/sbin/sshd [listener] 0 of 10-100 startups
sshd_config OK (root + key auth enabled)



### Task 4 – Create a 600 MB placeholder file on the host
Create **`~/bigfile.bin`** filled with zeroes.

In [26]:
# You do not have to do anything for task 4 except run this command
# Create a 600 MB test file in your temp directory
dd if=/dev/zero of=/tmp/bigfile.bin bs=1M count=600 status=progress
# change permissions
chmod 666 /tmp/bigfile.bin
# verification – should report 600 MB
ls -lh /tmp/bigfile.bin

600+0 records in
600+0 records out
629145600 bytes (629 MB, 600 MiB) copied, 0.549081 s, 1.1 GB/s
-rw-rw-rw- 1 root root 600M May  9 14:11 /tmp/bigfile.bin


### Task 5 – Authorise your host SSH key inside each container
Copy the contents of `/root/.ssh/id_ed25519.pub` from the host into `/root/.ssh/authorized_keys` in both containers so you can `ssh root@<container‑ip>` without a password.

In [27]:
# Generate a new ed25519 ssh key for root user on the host.
# ed25519 is considered the best overall encryption type for ssh keys
ssh-keygen -t ed25519 -f /root/.ssh/id_ed25519 -N "" -q

In [28]:
# your commands here (or do it in a terminal)
cat /root/.ssh/id_ed25519.pub

ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIP6ULsjoHLTx0c9cEaozm33UM3exGW2wKKIrhrAVOT8G root@CS447-alyssaaragon


In [1]:
# verification - host pub key should appear once in each container's authorized_keys file
# Correct output shows 1 for both containers. Incorrect shows 0.
pubkey=$(awk '{print $2}' /root/.ssh/id_ed25519.pub)
for c in debian1 debian2; do
  echo -n "$c: "
  sudo docker exec "$c" grep -c -- "$pubkey" /root/.ssh/authorized_keys
done

debian1: 1
debian2: 1


### Task 6 – Use Ansible to copy the big file into both containers
1. Find the IPs of the docker containers.
2. Verify key auth ssh access.
3. `apt install ansible` in a terminal.
4. Create an inventory file `/srv/ansible/hosts` with a group `[containers]` containing the two container IPs.
5. Write a one‑task playbook `/srv/ansible/copy_bigfile.yml` that copies `/tmp/bigfile.bin` to `/root/host/bigfile.bin` on each container.

After running the playbook the file should be present on **both** LVM‑backed filesystems.

In [2]:
# Find the IP addresses of both docker containers in a terminal
IP1="172.18.0.2" # debian1's IP goes in quotes here
IP2="172.18.0.3" # debian2's IP goes in quotes here

In [3]:
# verify ssh access using the IPs obtained above
(ssh -o StrictHostKeyChecking=accept-new root@$IP1 'echo Successfully SSHed to container 1.' && exit) || echo Failed to ssh to container 1.
(ssh -o StrictHostKeyChecking=accept-new root@$IP2 'echo Successfully SSHed to container 2.' && exit) || echo Failed to ssh to container 2.

Successfully SSHed to container 1.
Successfully SSHed to container 2.


In [4]:
# Write your commands here.
# You can do most of your commands in a terminal, however:
# you MUST run the ansible-playbook command in this cell to get credit for this part.
sudo ansible-playbook -i /srv/ansible/hosts /srv/ansible/copy_bigfile.yml



PLAY [Copy big file to containers] *********************************************

TASK [Gathering Facts] *********************************************************
ok: [debian1]
ok: [debian2]

TASK [Copy bigfile to container] ***********************************************
changed: [debian1]
changed: [debian2]

PLAY RECAP *********************************************************************
debian1                    : ok=2    changed=1    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   
debian2                    : ok=2    changed=1    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   



In [5]:
# verification – stat module should report 'exists: True' twice
# also verification of contents of the ansible hosts and task yaml
ansible containers -i /srv/ansible/hosts -m stat -a 'path=/root/host/bigfile.bin' | grep -E '(ok|exists)'
echo
echo
echo /srv/ansible/hosts
cat /srv/ansible/hosts
echo
echo
echo /srv/ansible/copy_bigfile.yml
cat /srv/ansible/copy_bigfile.yml

[WARNING]: Platform linux on host debian1 is using the discovered Python
interpreter at /usr/bin/python3.11, but future installation of another Python
interpreter could change the meaning of that path. See
https://docs.ansible.com/ansible-
core/2.18/reference_appendices/interpreter_discovery.html for more information.
[WARNING]: Platform linux on host debian2 is using the discovered Python
interpreter at /usr/bin/python3.11, but future installation of another Python
interpreter could change the meaning of that path. See
https://docs.ansible.com/ansible-
core/2.18/reference_appendices/interpreter_discovery.html for more information.
        "exists": true,
        "exists": true,


/srv/ansible/hosts
[containers]
debian1 ansible_host=172.18.0.2
debian2 ansible_host=172.18.0.3


/srv/ansible/copy_bigfile.yml
---
- name: Copy big file to containers
  hosts: containers
  tasks:
    - name: Copy bigfile to container
      copy:
        src: /tmp/bigfile.bin
        dest: /root/host/bigfile.

### Task 7 - Graduate (647) students only - Data recovery with LVM snapshots



With the 600 MB file still on both LVMs, you will:

1. Create a physical volume with your last remaining virtual disk.
2. Use `vgextend` to add the remaining virtual disk to `vgstripe1`.
3. Create a logical volume `lvbackup` that occupies 100% of `vgstripe1`'s remaining free space.
4. Create a read-only snapshots of `/dev/vgstripe1/lvm1` naming it `lvm1_snap` (allocate 600 MB).  
5. Print the SHA-256 checksum of the original source file `/tmp/bigfile.bin`.  
6. Simulate disaster by corrupting the copy on LV1 (`/mnt/lvm1/bigfile.bin`).
7. Print the SHA-256 checksum of the corrupted file `/mnt/lvm1/bigfile.bin`. 
8. Mount the corresponding snapshot as read-only, restore the good file, then unmount.
9. Confirm the file was restored successfully by printing its SHA-256 checksum.

In the cell below, write the commands to:
1. Create a pv from your last remaining 1GB virtual disk.
2. Extend `vgstripe1` with that virtual disk
3. Create a read-only lvm snapshot named `lvm1_snap` of `/dev/vgstripe1/lvm1` that occupies all the remaining space on `vgstripe1`

In [ ]:
# your commands here (or do it in a terminal)


In [ ]:
# Verification – /dev/sdf must be a PV in BOTH VGs
sudo pvs | grep -q '/dev/sdf' && echo "Success: /dev/sdf is a PV" || echo "Error: /dev/sdf not a PV"
sudo vgdisplay vgstripe1 -v | grep -q '/dev/sdf' && echo "Success: vgstripe1 uses /dev/sdf" || echo "Error: vgstripe1 missing PV"
sudo lvs | grep -E 'lvm1_snap' && echo "Success: snapshot present" || echo "Error: snapshot missing"

In [ ]:
# Capture original checksums (this might take a few seconds)
echo "Original hashes"
sha256sum /tmp/bigfile.bin
sha256sum /mnt/lvm1/bigfile.bin

In [ ]:
# Corrupt the file on lvm1 (simulate data loss) and calculate hashes again
sudo truncate -s 0 /mnt/lvm1/bigfile.bin
echo "Hashes after corrupting one file"
sha256sum /tmp/bigfile.bin
sha256sum /mnt/lvm1/bigfile.bin

In the cell below, mount `/dev/vgstripe1/lvm1_snap` to `/mnt/lvm1_snap`, then copy `/mnt/lvm1_snap/bigfile.bin` to `/mnt/lvm1/bigfile.bin`.

In [ ]:
# Your commands here.
# You must run the mount and copy commands in this to get gredit for this part.


In [ ]:
# Verify that hashes match again
echo "Hashes after restoring from snapshot"
sha256sum /tmp/bigfile.bin
sha256sum /mnt/lvm1/bigfile.bin